In [10]:
import pandas as pd
import joblib

In [241]:
df = pd.read_csv(r'C:/Users/Owner/OneDrive - 계명대학교/DC/2024/2024_날씨빅콘/electric_test.csv')

In [242]:
df

,NUM,TM,HH24,STN,nph_ta,nph_hm,nph_ws_10m,nph_rn_60m,nph_ta_chi,weekday,week_name,elect
0,4816,2023-01-01 01:00:00,1,752,3.0,68.6,2.9,0.0,-0.1,6,1.0,NaN
1,4816,2023-01-01 02:00:00,2,752,3.1,69.4,2.7,0.0,0.3,6,1.0,NaN
2,4816,2023-01-01 03:00:00,3,752,3.6,68.3,2.3,0.0,1.2,6,1.0,NaN
3,4816,2023-01-01 04:00:00,4,752,4.0,69.2,3.1,0.0,1.1,6,1.0,NaN
4,4816,2023-01-01 05:00:00,5,752,4.2,69.5,2.5,0.0,2.0,6,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2829473,12322,2023-12-31 20:00:00,20,901,5.6,70.8,3.2,0.0,5.8,6,1.0,NaN
2829474,12322,2023-12-31 21:00:00,21,901,5.3,69.1,3.6,0.0,4.5,6,1.0,NaN
2829475,12322,2023-12-31 22:00:00,22,901,5.1,70.6,3.0,0.0,5.1,6,1.0,NaN
2829476,12322,2023-12-31 23:00:00,23,901,5.2,69.7,3.8,0.0,5.1,6,1.0,NaN


In [243]:
df.rename(columns={'NUM': '격자넘버'}, inplace=True)
df.rename(columns={'TM': '날짜'}, inplace=True)
df.rename(columns={'HH24': '시간'}, inplace=True)
df.rename(columns={'STN': '지점번호'}, inplace=True)
df.rename(columns={'nph_ta': '기온'}, inplace=True)
df.rename(columns={'nph_hm': '상대습도'}, inplace=True)
df.rename(columns={'nph_ws_10m': '풍속'}, inplace=True)
df.rename(columns={'nph_rn_60m': '강수량'}, inplace=True)
df.rename(columns={'nph_ta_chi': '체감온도'}, inplace=True)
df.rename(columns={'weekday': '요일'}, inplace=True)
df.rename(columns={'week_name': '주중주말'}, inplace=True)

In [244]:
base_temp = 18.0

# CDH와 HDH 계산 함수
def calculate_cdh(temp, base_temp):
    return max(0, temp - base_temp)

def calculate_hdh(temp, base_temp):
    return max(0, base_temp - temp)

# df 셋
df['CDH'] = df.apply(lambda x: calculate_cdh(x['기온'], base_temp), axis=1)
df['HDH'] = df.apply(lambda x: calculate_hdh(x['기온'], base_temp), axis=1)

# 불쾌지수 컬럼 추가
def calculate_discomfort_index(temperature, humidity):
    return 1.8 * temperature - 0.55 * (1 - humidity / 100) * (1.8 * temperature - 26) + 32

# df 셋
df['불쾌지수'] = df.apply(lambda x: calculate_discomfort_index(x['기온'], x['상대습도']), axis=1)


In [245]:
df = df.sort_values(by='날짜')
df['날짜'] = pd.to_datetime(df['날짜'])

In [246]:
def assign_season(date):
    if (date.month == 2) or (date.month == 3) or (date.month == 4):
        return '봄'
    elif (date.month == 5) or (date.month == 6) or (date.month == 7):
        return '여름'
    elif (date.month == 8) or (date.month == 9) or (date.month == 10):
        return '가을'
    else:
        return '겨울'

# '계절' 컬럼 생성
df['계절'] = df['날짜'].apply(assign_season)

In [247]:
df

,격자넘버,날짜,시간,지점번호,기온,상대습도,풍속,강수량,체감온도,요일,주중주말,elect,CDH,HDH,불쾌지수,계절
0,4816,2023-01-01 01:00:00,1,752,3.0,68.6,2.9,0.0,-0.1,6,1.0,NaN,0.0,15.0,40.957620,겨울
2627998,18574,2023-01-01 01:00:00,1,876,2.1,48.3,0.5,0.0,2.1,6,1.0,NaN,0.0,15.9,42.098257,겨울
201480,18828,2023-01-01 01:00:00,1,511,0.2,54.3,1.9,0.0,-0.3,6,1.0,NaN,0.0,17.8,38.804614,겨울
1340280,17794,2023-01-01 01:00:00,1,436,-2.5,69.6,0.5,0.0,-4.1,6,1.0,NaN,0.0,20.5,32.599600,겨울
1489200,19136,2023-01-01 01:00:00,1,451,1.1,47.3,1.6,0.0,-2.5,6,1.0,NaN,0.0,16.9,40.942197,겨울
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2522877,11271,2024-01-01 00:00:00,24,296,1.3,64.3,0.5,0.0,1.3,0,0.0,NaN,0.0,16.7,38.985641,겨울
2006039,14239,2024-01-01 00:00:00,24,279,1.2,88.5,0.1,0.0,1.2,0,0.0,NaN,0.0,16.8,35.667880,겨울
814679,13619,2024-01-01 00:00:00,24,702,4.0,77.9,1.6,0.0,2.7,0,0.0,NaN,0.0,14.0,41.485140,겨울
840959,17791,2024-01-01 00:00:00,24,119,-1.2,96.0,0.0,0.0,-1.2,0,0.0,NaN,0.0,19.2,30.459520,겨울


In [248]:
seasonal_averages = df.groupby('계절').agg({
    '기온':'mean',
    '상대습도':'mean',
    '체감온도':'mean',
    '풍속':'mean',
    '강수량':'mean'
}).reset_index()

seasonal_averages

,계절,기온,상대습도,체감온도,풍속,강수량
0,가을,22.021828,78.373901,23.030252,1.543439,0.169337
1,겨울,5.729009,63.054185,2.239713,2.061995,0.074200
2,봄,9.466310,63.172411,8.188722,1.979861,0.057899
3,여름,21.210946,81.588478,24.043474,1.718530,0.303438


In [249]:
df = df.merge(seasonal_averages, on='계절', suffixes=('', '_계절별평균'))

In [250]:
def assign_date(df, date_column):
    df['년'] = df[date_column].dt.year
    df['월'] = df[date_column].dt.month
    df['일'] = df[date_column].dt.day
    df['시간'] = df[date_column].dt.hour
    return df

test = assign_date(df, '날짜')

In [251]:
df = pd.get_dummies(test, columns=['계절'])

In [252]:
df

,격자넘버,날짜,시간,지점번호,기온,상대습도,풍속,강수량,체감온도,요일,...,체감온도_계절별평균,풍속_계절별평균,강수량_계절별평균,년,월,일,계절_가을,계절_겨울,계절_봄,계절_여름
0,4816,2023-01-01 01:00:00,1,752,3.0,68.6,2.9,0.0,-0.1,6,...,2.239713,2.061995,0.0742,2023,1,1,False,True,False,False
1,18574,2023-01-01 01:00:00,1,876,2.1,48.3,0.5,0.0,2.1,6,...,2.239713,2.061995,0.0742,2023,1,1,False,True,False,False
2,18828,2023-01-01 01:00:00,1,511,0.2,54.3,1.9,0.0,-0.3,6,...,2.239713,2.061995,0.0742,2023,1,1,False,True,False,False
3,17794,2023-01-01 01:00:00,1,436,-2.5,69.6,0.5,0.0,-4.1,6,...,2.239713,2.061995,0.0742,2023,1,1,False,True,False,False
4,19136,2023-01-01 01:00:00,1,451,1.1,47.3,1.6,0.0,-2.5,6,...,2.239713,2.061995,0.0742,2023,1,1,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2829473,11271,2024-01-01 00:00:00,0,296,1.3,64.3,0.5,0.0,1.3,0,...,2.239713,2.061995,0.0742,2024,1,1,False,True,False,False
2829474,14239,2024-01-01 00:00:00,0,279,1.2,88.5,0.1,0.0,1.2,0,...,2.239713,2.061995,0.0742,2024,1,1,False,True,False,False
2829475,13619,2024-01-01 00:00:00,0,702,4.0,77.9,1.6,0.0,2.7,0,...,2.239713,2.061995,0.0742,2024,1,1,False,True,False,False
2829476,17791,2024-01-01 00:00:00,0,119,-1.2,96.0,0.0,0.0,-1.2,0,...,2.239713,2.061995,0.0742,2024,1,1,False,True,False,False


In [253]:
spring_va = df[df['계절_봄']==True]
summer_va = df[df['계절_여름']==True]
autum_va = df[df['계절_가을']==True]
winter_va = df[df['계절_겨울']==True]

In [254]:
loaded_model = joblib.load(r'C:/Users/Owner/OneDrive - 계명대학교/DC/2024/2024_날씨빅콘/spring_best_model.pkl')

c:\Users\Owner\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.4.2 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
c:\Users\Owner\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.4.2 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
c:\Users\Owner\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegre

In [255]:
spring_va

,격자넘버,날짜,시간,지점번호,기온,상대습도,풍속,강수량,체감온도,요일,...,체감온도_계절별평균,풍속_계절별평균,강수량_계절별평균,년,월,일,계절_가을,계절_겨울,계절_봄,계절_여름
239989,11870,2023-02-01 00:00:00,0,939,1.0,76.9,0.0,0.0,1.0,2,...,8.188722,1.979861,0.057899,2023,2,1,False,False,True,False
239990,17938,2023-02-01 00:00:00,0,545,2.3,70.2,0.3,0.0,4.3,2,...,8.188722,1.979861,0.057899,2023,2,1,False,False,True,False
239991,14091,2023-02-01 00:00:00,0,825,4.5,67.4,0.5,0.0,-2.1,2,...,8.188722,1.979861,0.057899,2023,2,1,False,False,True,False
239992,18539,2023-02-01 00:00:00,0,457,2.9,73.4,0.3,0.0,5.1,2,...,8.188722,1.979861,0.057899,2023,2,1,False,False,True,False
239993,10222,2023-02-01 00:00:00,0,294,9.4,79.3,1.4,0.0,3.0,2,...,8.188722,1.979861,0.057899,2023,2,1,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
929912,17794,2023-04-30 23:00:00,23,436,12.2,82.7,0.4,0.0,11.8,6,...,8.188722,1.979861,0.057899,2023,4,30,False,False,True,False
929913,10935,2023-04-30 23:00:00,23,158,13.1,86.0,4.5,0.0,11.5,6,...,8.188722,1.979861,0.057899,2023,4,30,False,False,True,False
929914,9884,2023-04-30 23:00:00,23,165,10.7,80.3,5.8,0.0,11.9,6,...,8.188722,1.979861,0.057899,2023,4,30,False,False,True,False
929915,18984,2023-04-30 23:00:00,23,424,12.9,64.9,2.9,0.0,11.8,6,...,8.188722,1.979861,0.057899,2023,4,30,False,False,True,False


In [256]:
predictions_new_data = loaded_model.predict(spring_va.drop(['년','날짜'],axis=1))

In [257]:
spring_va['elect'] = predictions_new_data

C:\Users\Owner\AppData\Local\Temp\ipykernel_20864\1141077533.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [258]:
loaded_model = joblib.load(r'C:/Users/Owner/OneDrive - 계명대학교/DC/2024/2024_날씨빅콘/summer_best_model.pkl')

c:\Users\Owner\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.4.2 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
c:\Users\Owner\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.4.2 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
c:\Users\Owner\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegre

In [259]:
predictions_new_data = loaded_model.predict(summer_va.drop(['년','날짜'],axis=1))

In [260]:
summer_va['elect'] = predictions_new_data

C:\Users\Owner\AppData\Local\Temp\ipykernel_20864\637462759.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [261]:
loaded_model = joblib.load(r'C:/Users/Owner/OneDrive - 계명대학교/DC/2024/2024_날씨빅콘/autum_best_model.pkl')

c:\Users\Owner\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.4.2 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
c:\Users\Owner\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.4.2 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
c:\Users\Owner\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegre

In [262]:
predictions_new_data = loaded_model.predict(autum_va.drop(['년','날짜'],axis=1))

In [263]:
autum_va['elect'] = predictions_new_data

C:\Users\Owner\AppData\Local\Temp\ipykernel_20864\2632530535.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [264]:
loaded_model = joblib.load(r'C:/Users/Owner/OneDrive - 계명대학교/DC/2024/2024_날씨빅콘/winter_best_model.pkl')

c:\Users\Owner\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.4.2 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
c:\Users\Owner\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.4.2 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
c:\Users\Owner\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegre

In [265]:
predictions_new_data = loaded_model.predict(winter_va.drop(['년','날짜'],axis=1))

In [266]:
winter_va['elect'] = predictions_new_data

C:\Users\Owner\AppData\Local\Temp\ipykernel_20864\202045210.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [267]:
concat2 = pd.concat([spring_va, summer_va, autum_va, winter_va], ignore_index=True)

In [268]:
concat2

,격자넘버,날짜,시간,지점번호,기온,상대습도,풍속,강수량,체감온도,요일,...,체감온도_계절별평균,풍속_계절별평균,강수량_계절별평균,년,월,일,계절_가을,계절_겨울,계절_봄,계절_여름
0,11870,2023-02-01,0,939,1.0,76.9,0.0,0.0,1.0,2,...,8.188722,1.979861,0.057899,2023,2,1,False,False,True,False
1,17938,2023-02-01,0,545,2.3,70.2,0.3,0.0,4.3,2,...,8.188722,1.979861,0.057899,2023,2,1,False,False,True,False
2,14091,2023-02-01,0,825,4.5,67.4,0.5,0.0,-2.1,2,...,8.188722,1.979861,0.057899,2023,2,1,False,False,True,False
3,18539,2023-02-01,0,457,2.9,73.4,0.3,0.0,5.1,2,...,8.188722,1.979861,0.057899,2023,2,1,False,False,True,False
4,10222,2023-02-01,0,294,9.4,79.3,1.4,0.0,3.0,2,...,8.188722,1.979861,0.057899,2023,2,1,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2829473,11271,2024-01-01,0,296,1.3,64.3,0.5,0.0,1.3,0,...,2.239713,2.061995,0.074200,2024,1,1,False,True,False,False
2829474,14239,2024-01-01,0,279,1.2,88.5,0.1,0.0,1.2,0,...,2.239713,2.061995,0.074200,2024,1,1,False,True,False,False
2829475,13619,2024-01-01,0,702,4.0,77.9,1.6,0.0,2.7,0,...,2.239713,2.061995,0.074200,2024,1,1,False,True,False,False
2829476,17791,2024-01-01,0,119,-1.2,96.0,0.0,0.0,-1.2,0,...,2.239713,2.061995,0.074200,2024,1,1,False,True,False,False


In [269]:
data = pd.read_csv(r'C:/Users/Owner/OneDrive - 계명대학교/DC/2024/2024_날씨빅콘/electric_test.csv')

In [270]:
concat2.to_csv(r'C:/Users/Owner/OneDrive - 계명대학교/DC/2024/2024_날씨빅콘/concat.csv')

In [271]:
data

,NUM,TM,HH24,STN,nph_ta,nph_hm,nph_ws_10m,nph_rn_60m,nph_ta_chi,weekday,week_name,elect
0,4816,2023-01-01 01:00:00,1,752,3.0,68.6,2.9,0.0,-0.1,6,1.0,NaN
1,4816,2023-01-01 02:00:00,2,752,3.1,69.4,2.7,0.0,0.3,6,1.0,NaN
2,4816,2023-01-01 03:00:00,3,752,3.6,68.3,2.3,0.0,1.2,6,1.0,NaN
3,4816,2023-01-01 04:00:00,4,752,4.0,69.2,3.1,0.0,1.1,6,1.0,NaN
4,4816,2023-01-01 05:00:00,5,752,4.2,69.5,2.5,0.0,2.0,6,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2829473,12322,2023-12-31 20:00:00,20,901,5.6,70.8,3.2,0.0,5.8,6,1.0,NaN
2829474,12322,2023-12-31 21:00:00,21,901,5.3,69.1,3.6,0.0,4.5,6,1.0,NaN
2829475,12322,2023-12-31 22:00:00,22,901,5.1,70.6,3.0,0.0,5.1,6,1.0,NaN
2829476,12322,2023-12-31 23:00:00,23,901,5.2,69.7,3.8,0.0,5.1,6,1.0,NaN


In [272]:
data['TM'] = pd.to_datetime(data['TM'])

In [273]:
data

,NUM,TM,HH24,STN,nph_ta,nph_hm,nph_ws_10m,nph_rn_60m,nph_ta_chi,weekday,week_name,elect
0,4816,2023-01-01 01:00:00,1,752,3.0,68.6,2.9,0.0,-0.1,6,1.0,NaN
1,4816,2023-01-01 02:00:00,2,752,3.1,69.4,2.7,0.0,0.3,6,1.0,NaN
2,4816,2023-01-01 03:00:00,3,752,3.6,68.3,2.3,0.0,1.2,6,1.0,NaN
3,4816,2023-01-01 04:00:00,4,752,4.0,69.2,3.1,0.0,1.1,6,1.0,NaN
4,4816,2023-01-01 05:00:00,5,752,4.2,69.5,2.5,0.0,2.0,6,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2829473,12322,2023-12-31 20:00:00,20,901,5.6,70.8,3.2,0.0,5.8,6,1.0,NaN
2829474,12322,2023-12-31 21:00:00,21,901,5.3,69.1,3.6,0.0,4.5,6,1.0,NaN
2829475,12322,2023-12-31 22:00:00,22,901,5.1,70.6,3.0,0.0,5.1,6,1.0,NaN
2829476,12322,2023-12-31 23:00:00,23,901,5.2,69.7,3.8,0.0,5.1,6,1.0,NaN


In [274]:
def assign_date(df, date_column):
    df['년'] = df[date_column].dt.year
    df['월'] = df[date_column].dt.month
    df['일'] = df[date_column].dt.day
    df['시간'] = df[date_column].dt.hour
    return df

test = assign_date(data, 'TM')

In [275]:
test

,NUM,TM,HH24,STN,nph_ta,nph_hm,nph_ws_10m,nph_rn_60m,nph_ta_chi,weekday,week_name,elect,년,월,일,시간
0,4816,2023-01-01 01:00:00,1,752,3.0,68.6,2.9,0.0,-0.1,6,1.0,NaN,2023,1,1,1
1,4816,2023-01-01 02:00:00,2,752,3.1,69.4,2.7,0.0,0.3,6,1.0,NaN,2023,1,1,2
2,4816,2023-01-01 03:00:00,3,752,3.6,68.3,2.3,0.0,1.2,6,1.0,NaN,2023,1,1,3
3,4816,2023-01-01 04:00:00,4,752,4.0,69.2,3.1,0.0,1.1,6,1.0,NaN,2023,1,1,4
4,4816,2023-01-01 05:00:00,5,752,4.2,69.5,2.5,0.0,2.0,6,1.0,NaN,2023,1,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2829473,12322,2023-12-31 20:00:00,20,901,5.6,70.8,3.2,0.0,5.8,6,1.0,NaN,2023,12,31,20
2829474,12322,2023-12-31 21:00:00,21,901,5.3,69.1,3.6,0.0,4.5,6,1.0,NaN,2023,12,31,21
2829475,12322,2023-12-31 22:00:00,22,901,5.1,70.6,3.0,0.0,5.1,6,1.0,NaN,2023,12,31,22
2829476,12322,2023-12-31 23:00:00,23,901,5.2,69.7,3.8,0.0,5.1,6,1.0,NaN,2023,12,31,23


In [277]:
concat2.columns

Index(['격자넘버', '날짜', '시간', '지점번호', '기온', '상대습도', '풍속', '강수량', '체감온도', '요일',
       '주중주말', 'elect', 'CDH', 'HDH', '불쾌지수', '기온_계절별평균', '상대습도_계절별평균',
       '체감온도_계절별평균', '풍속_계절별평균', '강수량_계절별평균', '년', '월', '일', '계절_가을', '계절_겨울',
       '계절_봄', '계절_여름'],
      dtype='object')

In [278]:
# 남기고 싶은 컬럼 리스트
columns_to_keep = ['년', '월','일','시간','elect']

# 특정 컬럼만 남기고 삭제
filtered_dataframe = concat2[columns_to_keep]

In [279]:
filtered_dataframe

,년,월,일,시간,elect
0,2023,2,1,0,109.995330
1,2023,2,1,0,112.776273
2,2023,2,1,0,104.707618
3,2023,2,1,0,109.384238
4,2023,2,1,0,106.236716
...,...,...,...,...,...
2829473,2024,1,1,0,110.836141
2829474,2024,1,1,0,113.700046
2829475,2024,1,1,0,112.180841
2829476,2024,1,1,0,112.638222


In [290]:
concat2.columns

Index(['격자넘버', '날짜', '시간', '지점번호', '기온', '상대습도', '풍속', '강수량', '체감온도', '요일',
       '주중주말', 'elect', 'CDH', 'HDH', '불쾌지수', '기온_계절별평균', '상대습도_계절별평균',
       '체감온도_계절별평균', '풍속_계절별평균', '강수량_계절별평균', '년', '월', '일', '계절_가을', '계절_겨울',
       '계절_봄', '계절_여름'],
      dtype='object')

In [292]:
concat2.drop(['CDH','HDH','불쾌지수','기온_계절별평균','상대습도_계절별평균','체감온도_계절별평균','풍속_계절별평균','강수량_계절별평균','계절_가을','계절_겨울','계절_여름','계절_봄'],axis=1, inplace=True)

In [296]:
a = ['년', '월', '일', '시간']

In [299]:
data

,NUM,TM,HH24,STN,nph_ta,nph_hm,nph_ws_10m,nph_rn_60m,nph_ta_chi,weekday,week_name,elect,년,월,일,시간
0,4816,2023-01-01 01:00:00,1,752,3.0,68.6,2.9,0.0,-0.1,6,1.0,NaN,2023,1,1,1
1,4816,2023-01-01 02:00:00,2,752,3.1,69.4,2.7,0.0,0.3,6,1.0,NaN,2023,1,1,2
2,4816,2023-01-01 03:00:00,3,752,3.6,68.3,2.3,0.0,1.2,6,1.0,NaN,2023,1,1,3
3,4816,2023-01-01 04:00:00,4,752,4.0,69.2,3.1,0.0,1.1,6,1.0,NaN,2023,1,1,4
4,4816,2023-01-01 05:00:00,5,752,4.2,69.5,2.5,0.0,2.0,6,1.0,NaN,2023,1,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2829473,12322,2023-12-31 20:00:00,20,901,5.6,70.8,3.2,0.0,5.8,6,1.0,NaN,2023,12,31,20
2829474,12322,2023-12-31 21:00:00,21,901,5.3,69.1,3.6,0.0,4.5,6,1.0,NaN,2023,12,31,21
2829475,12322,2023-12-31 22:00:00,22,901,5.1,70.6,3.0,0.0,5.1,6,1.0,NaN,2023,12,31,22
2829476,12322,2023-12-31 23:00:00,23,901,5.2,69.7,3.8,0.0,5.1,6,1.0,NaN,2023,12,31,23


In [300]:
concat2

,격자넘버,날짜,시간,지점번호,기온,상대습도,풍속,강수량,체감온도,요일,주중주말,elect,년,월,일
0,11870,2023-02-01,0,939,1.0,76.9,0.0,0.0,1.0,2,0.0,109.995330,2023,2,1
1,17938,2023-02-01,0,545,2.3,70.2,0.3,0.0,4.3,2,0.0,112.776273,2023,2,1
2,14091,2023-02-01,0,825,4.5,67.4,0.5,0.0,-2.1,2,0.0,104.707618,2023,2,1
3,18539,2023-02-01,0,457,2.9,73.4,0.3,0.0,5.1,2,0.0,109.384238,2023,2,1
4,10222,2023-02-01,0,294,9.4,79.3,1.4,0.0,3.0,2,0.0,106.236716,2023,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2829473,11271,2024-01-01,0,296,1.3,64.3,0.5,0.0,1.3,0,0.0,110.836141,2024,1,1
2829474,14239,2024-01-01,0,279,1.2,88.5,0.1,0.0,1.2,0,0.0,113.700046,2024,1,1
2829475,13619,2024-01-01,0,702,4.0,77.9,1.6,0.0,2.7,0,0.0,112.180841,2024,1,1
2829476,17791,2024-01-01,0,119,-1.2,96.0,0.0,0.0,-1.2,0,0.0,112.638222,2024,1,1


In [298]:

a = ['년', '월', '일', '시간']
merge_outer = pd.merge(data,concat2, how='inner',on=a)

MemoryError: Unable to allocate 6.81 GiB for an array with shape (913920750,) and data type int64

In [293]:
concat2

,격자넘버,날짜,시간,지점번호,기온,상대습도,풍속,강수량,체감온도,요일,주중주말,elect,년,월,일
0,11870,2023-02-01,0,939,1.0,76.9,0.0,0.0,1.0,2,0.0,109.995330,2023,2,1
1,17938,2023-02-01,0,545,2.3,70.2,0.3,0.0,4.3,2,0.0,112.776273,2023,2,1
2,14091,2023-02-01,0,825,4.5,67.4,0.5,0.0,-2.1,2,0.0,104.707618,2023,2,1
3,18539,2023-02-01,0,457,2.9,73.4,0.3,0.0,5.1,2,0.0,109.384238,2023,2,1
4,10222,2023-02-01,0,294,9.4,79.3,1.4,0.0,3.0,2,0.0,106.236716,2023,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2829473,11271,2024-01-01,0,296,1.3,64.3,0.5,0.0,1.3,0,0.0,110.836141,2024,1,1
2829474,14239,2024-01-01,0,279,1.2,88.5,0.1,0.0,1.2,0,0.0,113.700046,2024,1,1
2829475,13619,2024-01-01,0,702,4.0,77.9,1.6,0.0,2.7,0,0.0,112.180841,2024,1,1
2829476,17791,2024-01-01,0,119,-1.2,96.0,0.0,0.0,-1.2,0,0.0,112.638222,2024,1,1


In [280]:
data

,NUM,TM,HH24,STN,nph_ta,nph_hm,nph_ws_10m,nph_rn_60m,nph_ta_chi,weekday,week_name,elect,년,월,일,시간
0,4816,2023-01-01 01:00:00,1,752,3.0,68.6,2.9,0.0,-0.1,6,1.0,NaN,2023,1,1,1
1,4816,2023-01-01 02:00:00,2,752,3.1,69.4,2.7,0.0,0.3,6,1.0,NaN,2023,1,1,2
2,4816,2023-01-01 03:00:00,3,752,3.6,68.3,2.3,0.0,1.2,6,1.0,NaN,2023,1,1,3
3,4816,2023-01-01 04:00:00,4,752,4.0,69.2,3.1,0.0,1.1,6,1.0,NaN,2023,1,1,4
4,4816,2023-01-01 05:00:00,5,752,4.2,69.5,2.5,0.0,2.0,6,1.0,NaN,2023,1,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2829473,12322,2023-12-31 20:00:00,20,901,5.6,70.8,3.2,0.0,5.8,6,1.0,NaN,2023,12,31,20
2829474,12322,2023-12-31 21:00:00,21,901,5.3,69.1,3.6,0.0,4.5,6,1.0,NaN,2023,12,31,21
2829475,12322,2023-12-31 22:00:00,22,901,5.1,70.6,3.0,0.0,5.1,6,1.0,NaN,2023,12,31,22
2829476,12322,2023-12-31 23:00:00,23,901,5.2,69.7,3.8,0.0,5.1,6,1.0,NaN,2023,12,31,23
